Machine Learning Job Pair (Classification)
=======

## 1. Requirement
* python 2.x.x                                                         

* Tensorflow 2.0  

* CUDA 10.2  

* Cudnn >= 7.6.2  

* Pillow, SciPy, six, numpy, OpenCV 3.x.x

## 2. Environment  

* Ubuntu 18.04
* Nvidia-docker 2.0.3
* GeForce GTX1070 8G

### Data Augmentation  
* Traing 할수있는 데이터의 수를 늘리고자 회전, 가우시안 노이즈를 적용.

In [7]:
import csv, os
import cv2
import preprocess.augmentation as aug #가우시안 노이즈를 사용하기위한 외부소스

data_file_path = "/data/faces_images" # 데이터의 위치와 훈련데이터 정보가 있는 csv파일 경로
label_file_path = "/data/train_vision.csv" 

f = open('/data/noise_train_vision.csv','w')
wr = csv.writer(f)
wr.writerow(['filename','label'])

with open(label_file_path) as path_list:
    for data in enumerate(path_list):
        if data[0] is 0: continue
        filename, label = data[1].split(",")
        filepath = os.path.join(data_file_path,filename)
        
        ### 가우시안 노이즈 이미지 생성
        img = cv2.imread(filepath)
        noised_img = aug.gaussian_noise(img, mean=0, var =10) 
        
        ### Label파일 생성
        wr.writerow([filename, int(label)])
        cv2.imwrite(os.path.join(data_file_path,'noised_'+filename), noised_img)
        wr.writerow(['noised_'+filename, int(label)])
        
        '''
        for i in range(1,4):
            img = img.rotate(90)
            img.save(os.path.join(data_file_path,'rot'+str(90*i)+'_'+filename))
            wr.writerow(['rot'+str(90*i)+'_'+filename, int(label)])
        '''

### Data Preprocessing
* tensorflow keras의 데이터생성 함수의 경우 폴더 내부의 이미지를 읽어들여 Data Generator를 생성하기 때문에 이미지의 파일경로와 라벨을 가지고 데이터를 생성하는 flow_from_mapfile 구현.  
<br>
* Feature 잘 분류하기 위해 데이터 처리 과정에서 horizontal flip, zoom, shear, rescale 적용.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os, sys
import numpy as np
import preprocess.image as image
import tensorflow.keras.utils as utils
import tensorflow as tf
from tensorflow.compat.v1.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import GeneratorEnqueuer
from tensorflow.keras.utils import OrderedEnqueuer
print (tf.__version__)
import random


data_file_path = "/data/faces_images"
label_file_path = "/data/noise_train_vision.csv"
initial_learning_rate = 0.001 
image_size = 299
class_num = 6
batch_size = 16
epoch = 30

### GPU Memory 사용량을 제어하기 위한 코드 
### 기존의 tensorflow 1.x 버전의 Session을 이용하여 할당하는데 필요한 memory만을 사용.
'''
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.compat.v1.Session(config=config))
'''
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
# 프로그램 시작시에 메모리 증가가 설정되어야만 합니다

### 데이터 생성을 위한 path와 label리스트 생성
filenames = []
labels = []
with open(label_file_path) as path_list:
    for data in enumerate(path_list):
        if data[0] is 0:
            continue
            
        filename, label = data[1].split(",")
        filenames.append(os.path.join(data_file_path,filename))
        labels.append(int(label)-1)
        
    '''
    random.seed(1)    
    random.shuffle(filenames)
    random.shuffle(labels)
    '''
    
    print('Found %d images for training.' % (len(filenames)))
    
class_num = list(range(class_num))

'''
Class ImageDataGenerator
- Training에 사용되는 데이터를 생성하고 queue에 전달해주는 iterator.
  전달시 설정한 범위에서 random으로 filp, zoom, shearing을 수행하여 데이터를 보낸다.
'''
train_datagen = image.ImageDataGenerator(rescale=1. / 255,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True)

'''
Class DataPathIterator (기존의 DirectoryInterator를 수정)
- flow_from_mapfile의 return값. filename리스트를 읽어 데이터로 변환하며 random index를 통해 Training시 data batch
  를 random하게 구성하여 전달한다.

argment : 
        get_batches_of_transformed_samples : random index생성 및 이미지 로드를 하며 각 데이터 라벨에 대해 
        one-hot encording을 수행한다.
        
        next : 데이터의 다음 batch를 반환.
'''
train_generator = train_datagen.flow_from_mapfile(class_num,filenames,labels,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            class_mode='categorical')


2.0.0
Found 11700 images for training.


In [ ]:
import net.nets_factory as nets_factory
from tensorflow.keras import optimizers



#os.system("export TF_FORCE_GPU_ALLOW_GROWTH = true")
network_name = 'inception_v3'
save_model_path = "/home/june/model"
network_fn = nets_factory.get_network(network_name)
model = network_fn(include_top=True, weights=None, classes=len(class_num))


lr_schedule = optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                    decay_steps=40000,
                                                    decay_rate=0.96,
                                                    staircase=True)

#optimizer = optimizers.SGD(learning_rate = lr_schedule)
#optimizer = optimizers.RMSprop(learning_rate = lr_schedule)
optimizer = optimizers.Adam(learning_rate = lr_schedule, beta_1 = 0.9,
                            beta_2 = 0.999, epsilon = 1e-08, decay= 0.)
#optimizer = optimizers.Adagrad(learning_rate = lr_schedule)
model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

#model.load_weights(os.path.join(save_model_path,"inception_v3_10.ckpt"))

def create_data_queue(train_generator):
    is_sequence = isinstance(train_generator, Sequence)
    wait_time = 0.01
    if is_sequence:
        enqueuer = OrderedEnqueuer(train_generator,
                                use_multiprocessing=False,
                                shuffle = True)
    else:
        enqueuer = GeneratorEnqueuer(train_generator,
                                use_multiprocessing =False,
                                wait_time = wait_time)
    return enqueuer



enqueuer = create_data_queue(train_generator)
enqueuer.start(workers = 1, max_queue_size=10)
output_generator = enqueuer.get()
accuracy_mean = 0.
current_step = 0
current_epoch = 1
max_iter = len(filenames) / batch_size

try:
    while current_step < epoch * max_iter:
        if current_step%max_iter == 0:
            accuracy_mean = 0.
            current_epoch = int(current_step/max_iter)
            
            
        generator_output = next(output_generator)
        x, y = generator_output
        loss, accuracy = model.train_on_batch(x,y)            
        accuracy_mean = ((accuracy_mean * current_step-(max_iter*current_epoch))+accuracy) / (current_step-(max_iter*current_epoch)+1)
                
        sys.stdout.write('\r>> epoch : %d, loss : %f, accuracy : %f, step : %d/%d ' % (current_epoch, loss, accuracy, current_step, epoch * max_iter))
        sys.stdout.flush()
        current_step += 1
        
        if current_step%1000 ==0 :
            model.save_weights(os.path.join(save_model_path,'%s_%s_%d.ckpt'%('Adam',network_name,epoch)))
            
except Exception as ex:
    print(ex)
    enqueuer.stop(timeout=0.01)
    exit()
    
enqueuer.stop(timeout=0.01)
model.save_weights(os.path.join(save_model_path,'%s_%s_%d.ckpt'%('Adam',network_name,epoch)))
K.clear_session()

>> epoch : 4, loss : 1.024148, accuracy : 0.750000, step : 3112/21937 

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:62: RuntimeWarning: overflow encountered in double_scalars


>> epoch : 24, loss : 0.052964, accuracy : 1.000000, step : 20427/21937 

In [ ]:
%%javascript
Jupyter.notebook.session.delete();

In [2]:
test_file_path = "/data/test_vision.csv"
import net.nets_factory as nets_factory
from tensorflow.keras.preprocessing import image as kimage
import csv, os
import numpy as np
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow.compat.v1.keras import backend as K
import collections

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.compat.v1.Session(config=config))

def load_image(image_file_path):
        img = kimage.load_img(image_file_path, target_size=(image_size, image_size))
        x = kimage.img_to_array(img)
        x /= 255.
        return x
    
result = []
result2 = []
data_file_path = "/data/faces_images"
save_model_path = "/home/june/model"
network_fn = nets_factory.get_network('inception_resnet_v2')
network_fn2 = nets_factory.get_network('inception_resnet_v2')
model = network_fn(include_top=True, weights=None, classes=6)
model2 = network_fn2(include_top=True, weights=None, classes=6)
image_size = 299

lr_schedule = optimizers.schedules.ExponentialDecay(0.001,
                                                    decay_steps=15000,
                                                    decay_rate=0.96,
                                                    staircase=True)

#optimizer = optimizers.SGD(learning_rate = lr_schedule)
#optimizer = optimizers.Adagrad(learning_rate = lr_schedule)

optimizer = optimizers.Adam(learning_rate = lr_schedule, beta_1 = 0.9,
                            beta_2 = 0.999, epsilon = 1e-08, decay= 0.)

model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

model2.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])



model.load_weights(os.path.join(save_model_path,"Adam_inception_resnet_v2_20.ckpt"))
model2.load_weights(os.path.join(save_model_path,"Adam_inception_resnet_v2_20.ckpt"))
count = collections.Counter()
stack = []
with open(test_file_path) as path_list:
    for image_path in enumerate(path_list):
        if image_path[0] is 0 : continue
        img = load_image(os.path.join(data_file_path,image_path[1].split("\n")[0]))
        x = np.expand_dims(img, axis=0)
        softmax = model.predict(x, batch_size=1)
        softmax2 = model.predict(x, batch_size=1)
        result.append(np.asarray(softmax).argmax() + 1)
        result2.append(np.asarray(softmax).argmax() + 1)
        print(result)
        print(result2)
        
        exit()
#df = pd.DataFrame(result, columns=['label'])
#df.to_csv("/data/result.csv")
f = open('/data/result_adam_inception_res.csv','w')
wr = csv.writer(f)
wr.writerow(['prediction'])
for i in result:
    wr.writerow([i])
f.close()

[1]
[1]
[1, 4]
[1, 4]
[1, 4, 1]
[1, 4, 1]
[1, 4, 1, 1]
[1, 4, 1, 1]
[1, 4, 1, 1, 4]
[1, 4, 1, 1, 4]
[1, 4, 1, 1, 4, 4]
[1, 4, 1, 1, 4, 4]
[1, 4, 1, 1, 4, 4, 4]
[1, 4, 1, 1, 4, 4, 4]
[1, 4, 1, 1, 4, 4, 4, 5]
[1, 4, 1, 1, 4, 4, 4, 5]
[1, 4, 1, 1, 4, 4, 4, 5, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4, 2]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4, 2]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4, 2, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4, 2, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4, 2, 4, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4, 2, 4, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4, 2, 4, 4, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4, 2, 4, 4, 4]
[1, 4, 1, 1, 4, 4, 4, 5, 4, 1, 4, 5, 4, 2, 4, 4,

KeyboardInterrupt: 